In [1]:
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random
import matplotlib.pyplot as plt

# SVGD

In [2]:
from svgd import svgd

To accelerate the function `update` with `jit` we had to use the `static_argnums` argument. This is because `jit` only knows how to deal with functions that are 1) pure, that is no side effects, and 2) take only np arrays as inputs and outputs.

In effect, the `static_argnums` argument tells `jit` to ignore the callable input `logp`. Everytime it changes, `jit` recompiles.

# Sample
## Experiment 1: one-dim Gaussian mixture

### Experiment 1.1
Target distribution:
$$ p(x) = \mathcal N(x; 0, 1) $$
Initialization:
$$ q_0(x) = \mathcal N(x; -10, 1) $$
Kernel bandwidth:
$$ h = \text{med}^2 / \log n, $$
where med is the median of the $ n \choose 2$ pairwise distances between the points $x_1, \dots, x_n$. Important to note: the bandwidth here changes adaptively across iterations.

In [3]:
from jax.scipy.stats import norm
from svgd import kernel_param_update_rule

@jit
def logp(x):
    """
    IN: single scalar np array x. alternatively, [x] works too
    OUT: scalar logp(x)
    """
    return np.squeeze(np.sum(norm.logpdf(x, loc=0, scale=1)))

In [ ]:
n = 100
stepsize = 0.001
L = int(1 / stepsize / 3)

# generate data
key = random.PRNGKey(6)
x0 = random.normal(key, (n,1)) - 10

xout, log = svgd(x0, logp, stepsize, L, kernel_param=None, kernel_param_update_rule=kernel_param_update_rule)

## Diagnostics

In [ ]:
plt.figure(1)
plt.subplots_adjust(hspace=0.8)


plt.subplot(311) # 2 plots on 0th axis, 1 plot on 1th axis, plot nr 1 --> 211
plt.title("Particle mean")
plt.plot(log["particle_mean"])
plt.xlabel("Step")

plt.subplot(312)
plt.title("Particle variance")
plt.plot(log["particle_var"])

plt.subplot(313)
plt.title("Kernel parameter")
plt.plot(log["kernel_params"])


plt.figure(2)
_ = plt.hist(x0[:, 0])
_ = plt.hist(xout[:, 0])

Now let's see what speedup `jit` gets us.

In [ ]:
%timeit svgd(x0, logp, stepsize, L, kernel_param=None, kernel_param_update_rule=kernel_param_update_rule)

* With jitted `update`: 12.2 ms per loop
* Without jitted `update`: 1.08 s per loop

So that's a speedup of around 100x. Sweet!

### Experiment 1.2
Target distribution:
$$ p(x) = 1/3 \mathcal N(-2, 1) + 2/3 \mathcal N(2, 1) $$
Initialization:
$$ q_0(x) = \mathcal N(-10, 1) $$
Kernel bandwidth:
$$ h = \text{med}^2 / \log n, $$
where med is the median of the $ n \choose 2$ pairwise distances between the points $x_1, \dots, x_n$. Important to note: the bandwidth here changes adaptively across iterations.

In [ ]:
from jax.scipy.stats import norm
from svgd import kernel_param_update_rule

def p(x):
    """
    IN: single scalar np array x. alternatively, [x] works too
    OUT: scalar logp(x)
    """
    out = 1/3 * norm.pdf(x, loc=-2, scale=1) + 2/3 * norm.pdf(x, loc=2, scale=1)
    return np.squeeze(out) # to be able to take a gradient, output must be scalar

grid = np.arange(-5, 5, 0.1)
_ = plt.plot(grid, p(grid))

def logp(x):
    return np.log(p(x))

In [ ]:
# experiment:
n = 100
d = 1
stepsize = 0.001
L = int(1 / stepsize)

# generate data
key = random.PRNGKey(0)
x0 = random.normal(key, (n,d)) - 10

In [ ]:
xout, log = svgd(x0, logp, stepsize, L, kernel_param = None, kernel_param_update_rule=kernel_param_update_rule)

## Diagnostics

In [ ]:
plt.figure(1)
plt.subplots_adjust(hspace=1.2)


plt.subplot(311) # 2 plots on 0th axis, 1 plot on 1th axis, plot nr 1 --> 211
plt.title("Particle mean")
plt.plot(log["particle_mean"])
# plt.xlabel("Step")



plt.subplot(312)
plt.title("Particle variance")
plt.plot(log["particle_var"])

plt.subplot(313)
plt.title("Kernel parameter")
plt.plot(log["kernel_params"])

plt.figure(2)
_ = plt.hist(x0[:, 0])
_ = plt.hist(xout[:, 0])

In [ ]:
## test functions
mse1 = (np.mean(xout) - 2/3)**2
mse2 = (np.mean(xout**2) - 5)**2
print(mse1)
print(mse2)

# Experiment 2: Multi-dim Gaussian

### Experiment 2.1
Same as before, only now $X$ is $d$-dimensional.
#### Target distribution:
$$ p(x) = \mathcal N(x; \mu, \Sigma), \ \ \ \text { where we take } \mu = (1, \dots, 1)^T \text{ and } \Sigma = I. $$
#### Initialization:
$$ q_0(x) = \mathcal N(x; \mu_0, \Sigma_0), \ \ \ \text{ with } \mu = (-10, \dots, -10) \text{ and } \Sigma = I. $$

#### Kernel:
We'll use a natural generalization of the RBF kernel: $k(x, y) = \exp(-\frac{1}{2} (x - y)^T \sigma^{-1} (x - y))$. The easiest way to adaptively choose the kernel is to let $\Sigma = \text{diag}(h_1, \dots, h_d)$ be diagonal, in which case the kernel is given by $k(x, y) = \exp(-\frac{1}{2} \sum_{k=1}^d \frac{(x_k - y_k)^2}{h_k})$. Then choose the bandwidth parameter
$$ h_k = \text{med}_k^2 / \log n, $$
where med$_k$ is the $k$-th entry in the ($d$-dimensional) median of the $ n \choose 2$ pairwise distances between the points $x_1, \dots, x_n$.

In [ ]:
from jax.scipy.stats import norm

@jit
def logp(x):
    """
    Parameters: 
    * x: np array of shape (d,)
    
    Returns: 
    * scalar log(p(x)), where p(x) is multidim gaussian
    """
    assert x.ndim == 1
    return np.squeeze(np.sum(norm.logpdf(x, loc=0, scale=1)))

In [ ]:
from svgd import kernel_param_update_rule
vkernel_param_update_rule = vmap(kernel_param_update_rule, 1)

In [ ]:
n = 100
d = 3
stepsize = 0.001
L = int(1 / stepsize )

# generate data
key = random.PRNGKey(6)
x0 = random.normal(key, (n,d)) - 10

xout, log = svgd(x0, logp, stepsize, L, kernel_param=None, kernel_param_update_rule=vkernel_param_update_rule)
if "errors" in log.keys():
    print(log["errors"])

In [ ]:
import matplotlib as mpl

In [ ]:
mpl.rcParams['figure.figsize'] = [8.0, 8]

In [ ]:
plt.figure(1)
plt.subplots_adjust(hspace=1.2)
log["particle_mean"] = np.array(log["particle_mean"])
log["particle_var"] = np.array(log["particle_var"])
log["kernel_params"] = np.array(log["kernel_params"])

plt.subplot(311) # 2 plots on 0th axis, 1 plot on 1th axis, plot nr 1 --> 211
plt.title("Particle mean")
_ = plt.plot(log["particle_mean"])
# plt.xlabel("Step")

plt.subplot(312)
plt.title("Particle variance")
_ = plt.plot(log["particle_var"])

plt.subplot(313)
plt.title("Kernel parameter")
_ = plt.plot(log["kernel_params"])

mpl.rcParams['figure.figsize'] = [8.0, 4.8]
plt.figure(2)
_ = plt.hist(x0[:, 0])
_ = plt.hist(xout[:, 0])

In [ ]:
np.mean(xout, axis=0)

In [ ]:
np.var(xout, axis=0)

# Experiment 3: Bayesian logistic regression

In [ ]:
# number of samples
n = 500

First, $\alpha \sim \text{Gamma}(a, b)$.

In [ ]:
from jax.scipy.stats import gamma

# Generate regression parameters
a = 2 # shape
# b = 1 # rate parameter (1 / scale)

key = random.PRNGKey(0)
gamma_samples = random.gamma(key, a, shape=(1000,))

count, bins, ignored = plt.hist(gamma_samples, bins = 100, density=True)
y = gamma.pdf(bins, a)
_ = plt.plot(bins, y, 'r')

In [ ]:
_ = plt.plot(bins, 1 / y)

In [ ]:
key = random.PRNGKey(0)
alpha = random.gamma(key, a, shape=(1,))

In [ ]:
print(alpha)

Now, $p(\beta \ \vert \alpha) = \mathcal N (\beta;\ 0, \alpha^{-1})$.

In [ ]:
beta = random.normal(key, shape=(2,)) * 1 / np.sqrt(alpha)

Finally, $\log \frac{\pi_i}{1 - \pi_i} = \beta_0 + \beta_1 X_i$. In other words,
$$ \pi_i = \frac{\text{exp}(\beta_0 + \beta_1 X_i)}{1 - \text{exp}(\beta_0 + \beta_1 X_i)}.$$

In [ ]:
x = np.linspace(start=-10, stop=10, num=n)
pi = np.exp(beta[0] + beta[1] * x / (1 - np.exp(beta[0] + beta[1] * x)))

We can now sample the $Y_i$, which are distributed as
$$Y_i \ \vert X_i \sim \text{Bernoulli}(\pi_i).$$

In [ ]:
y = random.bernoulli(key, p=pi, shape=(n,))

### Bayesian inference
We now have a set of observations $\{(y_i, x_i)\}_{i=1, \dots, n}$, (that is, `x` and `y`) and a prior $p(\beta_0, \beta_1)$ over the logistic regression parameters.

The prior is given as follows. Choose $\beta_0, \beta_1$ to be independently identically distributed as 
$$p(\beta) = p(\beta \vert \alpha) \ p(\alpha),$$
where 
\begin{align}   
    p(\beta \vert \alpha) &= \mathcal N(\beta; 0, \alpha^{-1}) \\
    p(\alpha) &= \text{Gamma}(\alpha; a, b),
\end{align}
and $a, b$ are fixed parameters.



Then the posterior is given by
\begin{equation}
    p(\beta_0, \beta_1 \vert x, y) \propto p(y \vert x; \beta_0, \beta_1) p(\beta_0, \beta_1).
\end{equation}

This posterior is the target distribution for SVGD.

In [ ]:
from jax.scipy.stats import norm, gamma

In [ ]:
def p(beta, alpha, x, y, a, b):
    """
    IN:
    * beta: np array w/ shape (2,), regression parameters
    * alpha: scalar > 0, precision parameter for beta prior
    * x: np array of shape (n,)
    * y: np array of shape (n,) and type bool (bernoulli sample)
    * a, b: scalars > 0, parameters for gamma prior on alpha
    
    OUT:
    * scalar, unnormalized posterior pdf evaluated at beta and alpha: p(beta, alpha | x, y, a, b)
    """
    assert beta.shape[0] == 2
    pi = np.exp(beta[0] + beta[1] * x) / (1 - np.exp(beta[0] + beta[1] * x))
    
    p1 = pi * y + (1 - pi) * (1 - y)
    p1 = np.prod(p1) # likelihood of all x's
    p2 = np.prod(norm.pdf(beta, loc=0, scale=1/alpha))
    p3 = gamma.pdf(alpha, a)
    
    return p1 * p2 * p3

In [ ]:
pb = vmap(p, in_axes=(0, None, None, None, None, None))
pbb = vmap(pb, in_axes=(0, None, None, None, None, None))

In [ ]:
def logp(beta, alpha, x, y, a, b):
    """
    IN:
    * beta: np array w/ shape (2,), regression parameters
    * alpha: scalar > 0, precision parameter for beta prior
    * x: np array of shape (n,)
    * y: np array of shape (n,) and type bool (bernoulli sample)
    * a, b: scalars > 0, parameters for gamma prior on alpha
    
    OUT:
    * scalar, unnormalized posterior pdf evaluated at beta and alpha: p(beta, alpha | x, y, a, b)
    """
    assert beta.shape[0] == 2
    
    log_pi = beta[0] + beta[1] * x - np.log(1 + np.exp(beta[0] + beta[1] * x)) # array of length n
    one_minus_log_pi = np.log(1 + np.exp(beta[0] + beta[1] * x)) # array of length n
    
    log_p1 = y * log_pi + (1 - y) * one_minus_log_pi # array of length n
    log_p1 = np.sum(log_p1) # scalar
    
    log_p2 = np.sum(norm.logpdf(beta, loc=0, scale=1/alpha))
    log_p3 = gamma.logpdf(alpha, a)
    
    return log_p1 + log_p2 + log_p3

In [ ]:
logpb = vmap(logp, in_axes=(0, None, None, None, None, None))
logpbb = vmap(logpb, in_axes=(0, None, None, None, None, None))

### quick 3d plot of beta

In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [ ]:
# make data
num = 60
X = np.linspace(-7, 7, num)
Y = np.linspace(-7, 7, num)
X, Y = np.meshgrid(X, Y) # both shape (40, 40)
bet = np.array(list(zip(X, Y))).reshape(num, num, 2)

b = np.array(1)
print(logpbb(bet, alpha, x, y, a, b).shape)

Z = logpbb(bet, alpha, x, y, a, b)
Z = np.squeeze(Z)

In [ ]:
1 / alpha

In [ ]:
# plot
fig = plt.figure()
ax = fig.gca(projection='3d')
plt.xlabel("beta_0")
plt.ylabel("beta_1")
ax.set_zlabel("p(beta_0, beta_1, alpha)")

surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, linewidth=0, antialiased=False)